In [2]:
import os 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from collections import defaultdict

In [3]:
url = 'https://servicios.infoleg.gob.ar/infolegInternet/buscarNormas.do'

In [10]:
driver = webdriver.Firefox()

href_dic = defaultdict(list)
resultados = defaultdict(str)

for year in range(1880, 2000, 1):
    driver.get(url)
    dropdown = driver.find_element(By.NAME, 'tipoNorma')
    select = Select(dropdown)
    select.select_by_visible_text('Ley')
    
    dropdown = driver.find_element(By.NAME, 'diaPubDesde')
    dropdown.click()
    dropdown.send_keys("1")
    
    dropdown = driver.find_element(By.NAME, 'mesPubDesde')
    select = Select(dropdown)
    select.select_by_visible_text('Ene') 
    
    dropdown = driver.find_element(By.NAME, 'anioPubDesde')
    dropdown.click()
    dropdown.send_keys(year)
    
    if year == 2024:
        dropdown = driver.find_element(By.NAME, 'diaPubHasta')
        dropdown.click()
        dropdown.send_keys("7")
        
        dropdown = driver.find_element(By.NAME, 'mesPubHasta')
        select = Select(dropdown)
        select.select_by_visible_text('Nov') 
        
        dropdown = driver.find_element(By.NAME, 'anioPubHasta')
        dropdown.click()
        dropdown.send_keys(year)
    else:
        dropdown = driver.find_element(By.NAME, 'diaPubHasta')
        dropdown.click()
        dropdown.send_keys("31")
        
        dropdown = driver.find_element(By.NAME, 'mesPubHasta')
        select = Select(dropdown)
        select.select_by_visible_text('Dic') 
        
        dropdown = driver.find_element(By.NAME, 'anioPubHasta')
        dropdown.click()
        dropdown.send_keys(year)
    
    try: 
        search_button = driver.find_element(By.XPATH, '//*[@id="busquedasBot"]/input[1]')
        search_button.click()
        
        q_resultados = driver.find_element(By.XPATH, '//*[@id="paging_cantidad"]')
        resultados[year] = q_resultados.text
        
        for page in range(2, 2000, 1):
            try:
                links = driver.find_elements(By.XPATH, '/html/body/form/div/table/tbody/tr/td[1]/a')
                
                for link in links:
                    ley = link.text
                    href = link.get_attribute('href')
                    href_dic[ley] = href
                    
                next_button = driver.find_element(By.XPATH, f'//*[@id="paging"]/div/a[{page}]')
                next_button.click()
            except Exception as e:
                break
    except Exception as e:
        continue
        
driver.quit()

In [11]:
for key, value in resultados.items():
    print(value)

Cantidad de Normas Encontradas: 17 en 1 página.
Cantidad de Normas Encontradas: 95 en 2 páginas.
Cantidad de Normas Encontradas: 86 en 2 páginas.
Cantidad de Normas Encontradas: 58 en 2 páginas.
Cantidad de Normas Encontradas: 46 en 1 página.
Cantidad de Normas Encontradas: 77 en 2 páginas.
Cantidad de Normas Encontradas: 78 en 2 páginas.
Cantidad de Normas Encontradas: 52 en 2 páginas.
Cantidad de Normas Encontradas: 86 en 2 páginas.
Cantidad de Normas Encontradas: 115 en 3 páginas.
Cantidad de Normas Encontradas: 245 en 5 páginas.
Cantidad de Normas Encontradas: 314 en 7 páginas.
Cantidad de Normas Encontradas: 89 en 2 páginas.
Cantidad de Normas Encontradas: 159 en 4 páginas.
Cantidad de Normas Encontradas: 480 en 10 páginas.
Cantidad de Normas Encontradas: 543 en 11 páginas.
Cantidad de Normas Encontradas: 910 en 19 páginas.
Cantidad de Normas Encontradas: 135 en 3 páginas.
Cantidad de Normas Encontradas: 668 en 14 páginas.
Cantidad de Normas Encontradas: 300 en 6 páginas.
Cantidad

In [12]:
len(href_dic)

17942

In [13]:
import requests
import tqdm
import re
import html
from bs4 import BeautifulSoup

In [14]:
driver = webdriver.Firefox()

normas = defaultdict(list)
for ley, url in href_dic.items():
    try:
        driver.get(url)
        links = driver.find_elements(By.TAG_NAME, 'a')
        fecha = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div/p[1]/span').text
        for link in links:
            href = link.get_attribute('href')
            if href.endswith('norma.htm'):
            	normas[ley] = [fecha, href]
             
    except Exception as e:
        continue
            
driver.quit()

In [15]:
len(normas)

4735

In [17]:
CLEANR = re.compile('<.*?>')
for ley, (fecha, link) in tqdm.tqdm(normas.items()):
    req = requests.get(link)
    
    soup = BeautifulSoup(req.text, 'html.parser')
    text = soup.get_text(separator='\n')
    
    with open(f'./output/{ley}_texto_{fecha[-4:]}.txt', 'w', encoding='utf-8') as f:
        paragraphs = text.split('\n')
        filtered_paragraphs = [p for p in paragraphs if len(p.strip()) > 1] 
    
        for para in filtered_paragraphs:
            clean_text = re.sub(CLEANR, '', para)
            clean_text = html.unescape(clean_text)
            f.write(clean_text.strip() + '\n\n')


100%|██████████| 4735/4735 [08:44<00:00,  9.02it/s]  
